# Predict phonon DoS for new materials and evaluate their specific heat capacities

- `ComprehensiveEvaluation`: the function that evaluate phonon DoS and heat capacities with the input `*.cif` file
- `AtomEmbeddingAndSumLastLayer`: the model function

In [1]:
import glob
import torch
import torch_geometric
import torch_scatter

import e3nn
from e3nn import rs, o3
from e3nn.point.data_helpers import DataPeriodicNeighbors
from e3nn.networks import GatedConvParityNetwork
from e3nn.kernel_mod import Kernel
from e3nn.point.message_passing import Convolution

import pymatgen
from pymatgen.core.structure import Structure

import time, os
import datetime
import pickle
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import h5py
from class_evaluate_MPdata import ComprehensiveEvaluation, AtomEmbeddingAndSumLastLayer

torch.set_default_dtype(torch.float64)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print('torch device:' , device)

torch device: cuda


### Import model and trained model parameters

In [2]:
model_data = torch.load('models/200803-1018_len51max1000_fwin101ord3_trial_run_full_data.torch', map_location=device)

model = AtomEmbeddingAndSumLastLayer(model_data['state']['linear.weight'].shape[1], 
                                     model_data['state']['linear.weight'].shape[0], 
                                     GatedConvParityNetwork(**model_data['model_kwargs']))

model.load_state_dict(model_data['state'])
model.to(device)
model.eval()

AtomEmbeddingAndSumLastLayer(
  (linear): Linear(in_features=118, out_features=64, bias=True)
  (model): GatedConvParityNetwork(
    (layers): ModuleList(
      (0): ModuleList(
        (0): Convolution(
          (kernel): Kernel (64x0e -> 32x0e,32x0e,32x1o)
        )
        (1): GatedBlockParity (32x0e + 32x0e + 32x1o -> 32x0e,32x1o)
      )
      (1): ModuleList(
        (0): Convolution(
          (kernel): Kernel (32x0e,32x1o -> 32x0e,64x0e,32x1e,32x1o)
        )
        (1): GatedBlockParity (32x0e + 64x0e + 32x1e,32x1o -> 32x0e,32x1e,32x1o)
      )
      (2): Convolution(
        (kernel): Kernel (32x0e,32x1e,32x1o -> 51x0e)
      )
    )
  )
  (relu): ReLU()
)

### Import frequency points to evaluate heat capacities 

Please make sure to unzip the file `models/phdos_e3nn_len51max1000_fwin101ord3.zip`

In [3]:
with open('models/phdos_e3nn_len51max1000_fwin101ord3.pkl', 'rb') as f:
    data_dict = pickle.load(f)
phfre = data_dict['phfre']

Load cif file names and filter out those materials with more than 13 atoms inside the unit cells.

In [4]:
with open('models/cif_unique_files.pkl', 'rb') as f: 
    ciflist_dict = pickle.load(f)

cif_name = ciflist_dict.get('cif_name')
cif_id = ciflist_dict.get('cif_id')
num_sites = ciflist_dict.get('num_sites')

cif_name_suc = [cif_name[i] for i in range(len(cif_name)) if num_sites[i] <= 13]
cif_id_suc = [cif_id[i] for i in range(len(cif_id)) if num_sites[i] <= 13]

- `T_lst`: temperature list that you want to evaluate $C_{V}$ at
- `cif_path`: where you stored all downloaded `*.cif` files
- `h5_file`: the `*.h5` file that you want to store calculated phonon DoS and heat capacities


In [5]:
T_lst = [273.15, 293.15]
cif_path = 'data/mp_data/'
h5_file = 'data/phdos_maxsites13_2020Aug27.h5'

In [6]:
for i in range(0,len(cif_id_suc)):
    material_id = cif_id_suc[i]
    chunk_evaluation = ComprehensiveEvaluation([cif_name_suc[i]], model_data['model_kwargs'], cif_path=cif_path, chunk_id=material_id)
    chunk_evaluation.predict_phdos(chunk_evaluation.data,model,device=device)
    chunk_evaluation.cal_heatcap(chunk_evaluation.phdos,phfre.tolist(),T_lst,chunk_evaluation.structures)
    if os.path.exists(h5_file):
        with h5py.File(h5_file, 'a') as hf:
            hf["material_id"].resize((hf["material_id"].shape[0]+np.array([material_id])[None,:].shape[0]),axis=0)
            hf["material_id"][-np.array([material_id])[None,:].shape[0]:] = np.array([material_id])[None,:]
            hf["phdos_max1"].resize((hf["phdos_max1"].shape[0]+np.array(chunk_evaluation.phdos).shape[0]),axis=0)
            hf["phdos_max1"][-np.array(chunk_evaluation.phdos).shape[0]:] = np.array(chunk_evaluation.phdos)
            hf["phdos_norm"].resize((hf["phdos_norm"].shape[0]+np.array(chunk_evaluation.phdos_norm).shape[0]),axis=0)
            hf["phdos_norm"][-np.array(chunk_evaluation.phdos_norm).shape[0]:] = np.array(chunk_evaluation.phdos_norm)
            hf["heat_cap_mol"].resize((hf["heat_cap_mol"].shape[0]+np.array(chunk_evaluation.C_v_mol).shape[0]),axis=0)
            hf["heat_cap_mol"][-np.array(chunk_evaluation.C_v_mol).shape[0]:] = np.array(chunk_evaluation.C_v_mol)
            hf["heat_cap_kg"].resize((hf["heat_cap_kg"].shape[0]+np.array(chunk_evaluation.C_v_kg).shape[0]),axis=0)
            hf["heat_cap_kg"][-np.array(chunk_evaluation.C_v_kg).shape[0]:] = np.array(chunk_evaluation.C_v_kg)
            
            print("{}   Calculating mp-{}          ".format(i, cif_id_suc[i]), end="\r", flush=True)
    else:
        with h5py.File(h5_file, 'w') as hf:
            hf.create_dataset("material_id", data=np.array([material_id])[None,:],
                              compression="gzip", compression_opts=9, chunks=True, maxshape=(None,None))
            hf.create_dataset("phdos_max1", data=np.array(chunk_evaluation.phdos),
                              compression="gzip", compression_opts=9, chunks=True, maxshape=(None,None))
            hf.create_dataset("phdos_norm", data=np.array(chunk_evaluation.phdos_norm),
                              compression="gzip", compression_opts=9, chunks=True, maxshape=(None,None))
            hf.create_dataset("heat_cap_mol", data=np.array(chunk_evaluation.C_v_mol),
                              compression="gzip", compression_opts=9, chunks=True, maxshape=(None,None))
            hf.create_dataset("heat_cap_kg", data=np.array(chunk_evaluation.C_v_kg),
                              compression="gzip", compression_opts=9, chunks=True, maxshape=(None,None))
            hf.create_dataset("phfre", data=phfre,
                              compression="gzip", compression_opts=9, chunks=True, maxshape=(None))
            print("Created new h5py data\n")


KeyboardInterrupt: 